In [2]:
#imports
import pychemia
from pyprocar import ProcarSelect,ProcarParser,UtilsProcar
import pyprocar
import re
import numpy as np
import sys

This function parses each symmetry operation used by VASP.

In [4]:
def outcarParse_Operators(outcar):
    with open(outcar) as f:
        txt = f.readlines()

    for i,line in enumerate(txt):
        if 'irot' in line:
            begin_table = i+1
        if 'Subroutine' in line:
            end_table = i-1

    operators = np.zeros((end_table-begin_table, 9))
    for i,line in enumerate(txt[begin_table:end_table]):
        str_list = line.split()
        num_list = [float(s) for s in str_list]
        operator = np.array(num_list)
        operators[i,:] = operator
    
    return operators

This converts the symmetry operations from the form of $\begin{pmatrix}u_{x}\\u_{y}\\u_{z}\end{pmatrix}$, $\theta$, det(A) to rotation matrices.

In [5]:
#finds rotation matrix from axis vector (normalized) and angle
def findR(operator):
    det_A = operator[1]
    #convert alpha to radians
    alpha = np.pi*operator[2]/180.
    #get rotation axis
    x = operator[3]
    y = operator[4]
    z = operator[5]
    
    R = np.array([
        [np.cos(alpha) + x**2*(1-np.cos(alpha)), x*y*(1-np.cos(alpha))-z*np.sin(alpha), x*z*(1-np.cos(alpha)) + y*np.sin(alpha)],
        [y*x*(1-np.cos(alpha)) + z*np.sin(alpha), np.cos(alpha) + y**2*(1-np.cos(alpha)), y*z*(1-np.cos(alpha)) - x*np.sin(alpha)],
        [z*x*(1-np.cos(alpha)) - y*np.sin(alpha), z*y*(1-np.cos(alpha)) + x*np.sin(alpha), np.cos(alpha) + z**2*(1-np.cos(alpha))]
    ])*det_A
    
    R = np.round_(R, decimals=3)
    
    #get translation
    #t = np.array(operator[6:])
    
    return R

This function applies each symmetry operation to each kpoint.

In [6]:
def just_apply_symmetries(kpoints):
    klist = frac_kpoints_sym.tolist()
    #for each symmetry operation
    for i,_ in enumerate(rotations): 
        #RU = np.matmul(rotations[i],np.transpose(reciprocal_lattice))
        #for each point
        for point in kpoints:
            #apply symmetry operation to kpoint
            sympoint_vector = np.dot(rotations[i], point)
            sympoint = sympoint_vector.tolist()
            klist.append(sympoint)
    new_kpoints = np.array(klist)
    return new_kpoints

Call the files and get the symmetry operations.

In [7]:
#get symops
procar_nosym = 'PROCAR_Si_nosym'
procar_sym = 'PROCAR_Si_sym'

operators = outcarParse_Operators('OUTCAR_Si')
rotations = np.array([findR(operator) for operator in operators])

"""
rf = open('OUTCAR','r')
outcar = rf.read()
rf.close()


structure = pychemia.code.vasp.read_poscar('POSCAR_Al')
cs = pychemia.crystal.CrystalSymmetry(structure)
#3x3 matrices to multiply
rotations = cs.get_symmetry()['rotations']

#3x1 vector to add
translations = cs.get_symmetry()['translations']
symmetry_vasp = re.findall('point\s*symmetry\s*([A-Z][_a-z0-9]*)',outcar)[0].strip().replace('_','') # THis is the symmetry recognized by the dft code
symmetry_spglib  = cs.get_space_group_type()['pointgroup_international'] # symmetry found by spglib
# These two should be the same symmetry_vasp,symmetry_spg. We might want to check in the future
print(symmetry_vasp,symmetry_spglib)
"""

"\nrf = open('OUTCAR','r')\noutcar = rf.read()\nrf.close()\n\n\nstructure = pychemia.code.vasp.read_poscar('POSCAR_Al')\ncs = pychemia.crystal.CrystalSymmetry(structure)\n#3x3 matrices to multiply\nrotations = cs.get_symmetry()['rotations']\n\n#3x1 vector to add\ntranslations = cs.get_symmetry()['translations']\nsymmetry_vasp = re.findall('point\\s*symmetry\\s*([A-Z][_a-z0-9]*)',outcar)[0].strip().replace('_','') # THis is the symmetry recognized by the dft code\nsymmetry_spglib  = cs.get_space_group_type()['pointgroup_international'] # symmetry found by spglib\n# These two should be the same symmetry_vasp,symmetry_spg. We might want to check in the future\nprint(symmetry_vasp,symmetry_spglib)\n"

In [8]:
np.set_printoptions(threshold=3)
print(rotations)

[[[ 1.  0.  0.]
  [ 0.  1. -0.]
  [ 0.  0.  1.]]

 [[-0.  1. -0.]
  [-0. -0.  1.]
  [ 1. -0. -0.]]

 [[-0. -0.  1.]
  [ 1. -0. -0.]
  [-0.  1. -0.]]

 ...

 [[-1. -0. -0.]
  [ 0.  0. -1.]
  [ 0. -1.  0.]]

 [[ 0. -1. -0.]
  [-1.  0. -0.]
  [ 0.  0. -1.]]

 [[ 0.  0. -1.]
  [-0. -1. -0.]
  [-1.  0.  0.]]]


This cell will check if a certain matrix listed on the BCS was calculated with our function. For Si it turns out that the matrices match perfectly.

In [9]:
bilbao_matrix = np.array([
    [0,0,1],
    [0,1,0],
    [1,0,0]
])
bilbao_check = np.any(np.array([np.all(R == bilbao_matrix) for R in rotations]))
print(bilbao_check)

True


Parse the kpoints from the VASP runs with and without symmetry.

In [10]:
#get kpoints
"""
outcarparser = UtilsProcar()

#parse reciprocal lattice matrix
reciprocal_lattice = np.transpose(outcarparser.RecLatOutcar('OUTCAR'))
print(reciprocal_lattice)

"""
procarFile = ProcarParser()
procarFile.readFile(procar_nosym, False)
data_nosym = ProcarSelect(procarFile, deepCopy=True)
frac_kpoints_nosym = data_nosym.kpoints
print(frac_kpoints_nosym)

procarFile = ProcarParser()
procarFile.readFile(procar_sym, False)
data_sym = ProcarSelect(procarFile, deepCopy=True)
frac_kpoints_sym = data_sym.kpoints
print(frac_kpoints_sym)

spd shape      :  (729, 32, 1, 3, 11) [kpoints, bands, spins, atoms+1, orbitals+2]
[[ 0.          0.          0.        ]
 [ 0.11111111  0.          0.        ]
 [ 0.22222222  0.          0.        ]
 ...
 [-0.33333333 -0.11111111 -0.11111111]
 [-0.22222222 -0.11111111 -0.11111111]
 [-0.11111111 -0.11111111 -0.11111111]]
spd shape      :  (35, 16, 1, 3, 11) [kpoints, bands, spins, atoms+1, orbitals+2]
[[ 0.          0.          0.        ]
 [ 0.11111111  0.          0.        ]
 [ 0.22222222  0.          0.        ]
 ...
 [-0.44444444  0.44444444  0.11111111]
 [-0.33333333  0.44444444  0.11111111]
 [-0.33333333  0.44444444  0.22222222]]


Apply symmetry operations and boundary conditions... remove repeated points.

In [11]:
kpoints_calculated_frac = just_apply_symmetries(frac_kpoints_sym)
#kpoints_calculated_frac = np.dot(kpoints_calculated, np.linalg.inv(reciprocal_lattice))
bound_ops = -1.0*(kpoints_calculated_frac > 0.5) + 1.0*(kpoints_calculated_frac < -0.5)
kpoints_calculated_frac += bound_ops
kpoints_unique = np.unique(kpoints_calculated_frac, axis=0)

In [12]:
print(kpoints_unique, kpoints_unique.shape)

[[-0.44444444 -0.44444444 -0.11111111]
 [-0.44444444 -0.44444444  0.        ]
 [-0.44444444 -0.44444444  0.11111111]
 ...
 [ 0.44444444  0.44444444 -0.11111111]
 [ 0.44444444  0.44444444  0.        ]
 [ 0.44444444  0.44444444  0.11111111]] (457, 3)


This counts how many incorrect points were found (false positives) and how many correct points were missed (misses) to help troubleshoot. 

In [13]:
#check errors
misses = 0
false_positives = 0
for point in frac_kpoints_nosym.tolist():
    if point not in kpoints_unique.tolist():
        misses += 1
        
for point in kpoints_unique.tolist():
    if point not in frac_kpoints_nosym.tolist():
        false_positives += 1

In [14]:
print(misses, false_positives)

272 0
